이전 글에서 Jupyter를 설치하고 Pelican을 이용하여 블로깅에 사용할수 있는 방법에 대해알아봤습니다.
[Jupyter-From-Dev-To-Blogging](https://comafire.github.io/posts/2016-Jupyter-From-Dev-To-Blogging)

이제 Jupyter 에서 로컬 노트북상에 Apache Spark을 사용할 수 있도록 연동해 보도록 하겠습니다. 

<!-- TEASER_END -->

### Apache Spark

Spark으로 무엇을 해보려면 클러스터가 있어야 한다는 선입견에 빠지기 쉽습니다만, 요즘 보편화 되어 있는 멀티 코어 노트북의 장점을 복잡한 멀티코어, 멀티스레드 프로그래밍 없이 적극 활용가능하다는 측면에서 클러스터가 아닌 로컬 노트북상에서도 충분한 활용성이 있습니다.

먼저, Java 가 설치되어 있는지 확인합니다. 

```bash
> which java
/usr/bin/java
> java -version
java version "1.8.0_91"
Java(TM) SE Runtime Environment (build 1.8.0_91-b14)
Java HotSpot(TM) 64-Bit Server VM (build 25.91-b14, mixed mode)
```

만약 설치가 안되어 있으면  http://www.oracle.com/technetwork/java/javase/downloads/index.html 오라클 홈페이지 상에서 다운받아 설치합니다.


다음으로 brew 를 이용하여 scala 를 설치합니다.

```bash
> brew install scala
```

이제 Spark을 설치합니다. https://spark.apache.org/downloads.html 사이트에서 원하는 버전의 Spark 을 다운로드 받습니다.

로컬에 Spark을 설치하는 것은 간단히 Spark을 받아 압축을 푸는 것으로 끝납니다. 편의를 위해서 심볼릭 링크를 걸어둡니다.

```bash
> cd /usr/local
> mv ~/Downloads/spark-2.0.0-bin-hadoop2.7.tgz ./
> tar -zxvf spark-2.0.0-bin-hadoop2.7.tgz
> ln -s spark-2.0.0-bin-hadoop2.7 spark
```

이제 쉘 환경설정파일에 환경변수를 설정합니다.

```
> vi ~/.bash_profile

export PATH="/usr/local/sbin:$PATH"

export JAVA_HOME=$(/usr/libexec/java_home)
export SCALA_HOME=/usr/local/bin/scala
export PATH=$PATH:$SCALA_HOME/bin

export SPARK_HOME=/usr/local/spark
export PATH=$PATH:$SPARK_HOME/bin
export PYTHONPATH=$SPARK_HOME/python/:$PYTHONPATH
export PYTHONPATH=$SPARK_HOME/python/lib/py4j-0.10-src.zip:$PYTHONPATH

export PYTHON=/usr/local/bin/python
```
py4j 라이브러리의 경우 설치한 spark 안에 라이브러리 버전을 확인하여 이름을 넣어주세요.


이제 Jupyter 와 연동하기 위해 Apache Toree를 설치합니다. 기존에 Jupyter와 Spark을 연결하는 방식이 좀 복잡했는데 Toree 를 이용하면 간단하게 Jupyter 와 연동이 가능합니다.

```bash
> pip install --pre toree
> jupyter toree install
```

이제 지금 블로깅하고 있는 Jupyter의 Kernel을 Toree리로 변경하고 간단히 로컬에 설치된 Spark를 테스트해보겠습니다.

먼저 Scala를 통해 Spark소스안에 포함된 예제 json 파일을 dataframe으로 읽어보겠습니다.

In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SQLContext

val df = sqlContext.read.json("/usr/local/spark/examples/src/main/resources/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



다음으로 Toree의 Magics 통해 Python과 R로 Spark을 사용해보겠습니다.

In [2]:
%%PySpark
df = sqlContext.read.json("/usr/local/spark/examples/src/main/resources/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



이 외에도 다양한 Magics를 지원합니다. 더 자세한 사용법은 아래 링크를 참조하세요.
https://github.com/apache/incubator-toree/blob/master/etc/examples/notebooks/magic-tutorial.ipynb

In [4]:
%%SparkR
df <- jsonFile(sqlContext, "/usr/local/spark/examples/src/main/resources/people.json")
showDF(df)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

한가지 아쉬운 점은 Toree 내에서 Python과 R의 경우 코드 자동완성 기능이 Spark함수 밖에 안된다는 것입니다.

Python과 R의 경우 따로 Python과 R커널에서 Spark을 띄워서 사용하면 Spark외 함수에 대한 코드 자동완성 기능을 사용할 수 있습니다. 